In [4]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time

from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import Angle, SkyCoord
import astropy.units as u
import geoviews
import pandas as pd
import datashader
import colorcet as cc
from cartopy import crs
from astropy.time import Time
from datetime import datetime

import holoviews as hv
import holoviews.operation.datashader as hd
from holoviews.operation.datashader import rasterize, shade, spread, dynspread, datashade
from holoviews.operation import decimate
import datashader
import pandas as pd
import numpy as np
import datashader as ds
from holoviews import opts

import hvplot.pandas
import bokeh
from bokeh.models import ColumnDataSource, HoverTool
from holoviews.selection import link_selections
from holoviews.plotting.links import DataLink
import param
from tqdm import tqdm

import colorcet as cc

In [5]:
# read the file

# filename = "/users/elowe/repos/antenna-plots/ant_pos_17AB_plus.parquet"
filename = "/users/elowe/repos/antenna-plots/modified_ant_pos_17AB_plus_date.parquet"
df = pd.read_parquet(filename)
# df = df[::100]
df

,DMJD,RAJ2000,DECJ2000,MNT_AZ,MNT_EL,Scan,Session,Receiver,Backend,date
index,,,,,,,,,,
0,57736.577778,253.582806,5.720351,119.852826,39.995771,1,AGBT17A_012_03,Rcvr1_2,DCR,2016-12-14 13:52:00
1,57736.577789,253.530629,5.671967,119.949294,39.998545,1,AGBT17A_012_03,Rcvr1_2,DCR,2016-12-14 13:52:01
2,57736.577801,253.478301,5.623275,120.046186,40.001152,1,AGBT17A_012_03,Rcvr1_2,DCR,2016-12-14 13:52:02
3,57736.577812,253.425677,5.574749,120.143223,40.003952,1,AGBT17A_012_03,Rcvr1_2,DCR,2016-12-14 13:52:03
4,57736.577824,253.372812,5.526043,120.240662,40.006741,1,AGBT17A_012_03,Rcvr1_2,DCR,2016-12-14 13:52:04
...,...,...,...,...,...,...,...,...,...,...
198501,58777.618438,140.807189,20.147786,244.928816,57.318839,1565,AGBT17B_325_205,Rcvr_342,GUPPI,2019-10-21 14:50:33
198502,58777.618449,140.807236,20.147763,244.933478,57.315894,1565,AGBT17B_325_205,Rcvr_342,GUPPI,2019-10-21 14:50:34
198503,58777.618461,140.807226,20.147874,244.938392,57.312998,1565,AGBT17B_325_205,Rcvr_342,GUPPI,2019-10-21 14:50:35


In [ ]:
# code to convert the DMJD to date
date = Time(df['DMJD'], format="mjd").to_datetime()
df['date'] = date
df

In [3]:
 %%time
sc = SkyCoord(df['RAJ2000'], df['DECJ2000'], unit=u.deg, frame="icrs", equinox="J2000")
sc

CPU times: user 26min 43s, sys: 56.5 s, total: 27min 40s
Wall time: 27min 52s


<SkyCoord (ICRS): (ra, dec) in deg
    [(253.58280557,  5.72035056), (253.53062857,  5.67196702),
     (253.47830086,  5.62327479), ..., (140.80722628, 20.1478745 ),
     (140.80708654, 20.14780753), (140.80718968, 20.14791216)]>

In [6]:
%%time

# Get skycoord into a dataframe
galactic_df = sc.galactic.to_table().to_pandas()
galactic_df.tail()

CPU times: user 18.1 s, sys: 3.23 s, total: 21.3 s
Wall time: 21.5 s


,l,b
22081176,209.584212,42.133570
22081177,209.584261,42.133604
22081178,209.584115,42.133631
22081179,209.584143,42.133486
22081180,209.584052,42.133611


In [8]:
# Creating one file with all the columns we need



gal_ra = galactic_df['l'].values.tolist()
gal_dec = galactic_df['b'].values.tolist()

df["gal_ra"] = gal_ra
df["gal_dec"] = gal_dec
df1 = df

# antenna_data2.to_parquet('modified_ant_pos_with_az_al_small.parquet', engine='fastparquet')
df1

,DMJD,RAJ2000,DECJ2000,MNT_AZ,MNT_EL,Scan,Session,Receiver,Backend,date,gal_ra,gal_dec
index,,,,,,,,,,,,
0,57736.577778,253.582806,5.720351,119.852826,39.995771,1,AGBT17A_012_03,Rcvr1_2,DCR,2016-12-14 13:52:00,24.208767,28.588167
1,57736.577789,253.530629,5.671967,119.949294,39.998545,1,AGBT17A_012_03,Rcvr1_2,DCR,2016-12-14 13:52:01,24.132405,28.611441
2,57736.577801,253.478301,5.623275,120.046186,40.001152,1,AGBT17A_012_03,Rcvr1_2,DCR,2016-12-14 13:52:02,24.055619,28.634669
3,57736.577812,253.425677,5.574749,120.143223,40.003952,1,AGBT17A_012_03,Rcvr1_2,DCR,2016-12-14 13:52:03,23.978810,28.658202
4,57736.577824,253.372812,5.526043,120.240662,40.006741,1,AGBT17A_012_03,Rcvr1_2,DCR,2016-12-14 13:52:04,23.901656,28.681830
...,...,...,...,...,...,...,...,...,...,...,...,...
198501,58777.618438,140.807189,20.147786,244.928816,57.318839,1565,AGBT17B_325_205,Rcvr_342,GUPPI,2019-10-21 14:50:33,209.584212,42.133570
198502,58777.618449,140.807236,20.147763,244.933478,57.315894,1565,AGBT17B_325_205,Rcvr_342,GUPPI,2019-10-21 14:50:34,209.584261,42.133604
198503,58777.618461,140.807226,20.147874,244.938392,57.312998,1565,AGBT17B_325_205,Rcvr_342,GUPPI,2019-10-21 14:50:35,209.584115,42.133631


In [10]:
df1.to_parquet('modified_ant_pos_17AB_plus_gal.parquet', engine='pyarrow')

In [11]:
from holoviews import opts
proj = geoviews.Points(df,['RAJ2000', 'DECJ2000'])
proj = proj.opts(opts.Points(tools=['lasso_select','box_select'],projection=crs.Mollweide(),global_extent=True, width=800, height=400))
proj = geoviews.operation.project_points(proj, projection=crs.Mollweide())

# creating the decimated points
# decimated_points = decimate(proj).opts(tools=['lasso_select', 'box_select', 'hover'], projection=crs.Mollweide(), )

hv_shaded = hv.operation.datashader.datashade(rasterize(hv.DynamicMap(proj)), dynamic=True, precompute=True, cmap='fire')
hv_spread = hv.operation.datashader.dynspread(hv_shaded, threshold=.40, max_px=1)
hv_spread = hv_spread.opts(tools=['lasso_select','help'],bgcolor='white',projection=crs.Mollweide(), global_extent=True, frame_width=800, frame_height=400)

In [ ]:
hv_spread